In [2]:
import numpy as np

leftList = np.load("/home/wngys/lab/DeepFold/protein_infor/leftIDArray.npy", allow_pickle=True).tolist()
toDeleList = np.load("/home/wngys/lab/DeepFold/protein_infor/negLess10IDArray.npy", allow_pickle=True).tolist()

toSelecList = list(set(leftList) - set(toDeleList))

In [4]:
import torch
import numpy as np

toSelecNum = len(toSelecList) # 14018
perm = torch.randperm(toSelecNum)

# 训练集 验证集 测试集 比例 6:2:2
train_indices = perm[:int(toSelecNum*0.6)]   # 8410
valid_indices = perm[int(toSelecNum*0.6):int(toSelecNum*0.8)] # 2804
test_indices = perm[int(toSelecNum*0.8):toSelecNum] # 2804

toSelecTrainList = [toSelecList[i] for i in train_indices]
toSelecValidList = [toSelecList[i] for i in valid_indices]
toSelecTestList = [toSelecList[i] for i in test_indices]

np.save("/home/wngys/lab/DeepFold/pair/train.npy", toSelecTrainList)
np.save("/home/wngys/lab/DeepFold/pair/valid.npy", toSelecValidList)
np.save("/home/wngys/lab/DeepFold/pair/test.npy", toSelecTestList)



In [2]:
import numpy as np
theSelecTrainList = np.load("/home/wngys/lab/DeepFold/pair/train.npy", allow_pickle=True)
# valid 
# test

In [3]:
# 更改数据分布 确保每个batch至少一个正例
import math
import copy

Dir = "/home/wngys/lab/DeepFold/pair/pair_bool_90/"
newDir = "/home/wngys/lab/DeepFold/pair/train_pair_bool_90/"
batch_size = 64

# print(theSelecTrainList)
idx = 0
for id in theSelecTrainList:
    idx += 1
    if idx % 100 == 0:
        print("idx: ", idx)
    posi_data = []
    nega_data = []

    with open(Dir+id+".txt", "r") as f_r:
        while True:
            lines = f_r.readline()
            if not lines:
                break
            # id2 = lines.split('\t')[0]
            flag = lines.split('\t')[1].split("\n")[0]
            # print(type(flag))
            if flag == "1":
                posi_data.append(lines)
            else:
                nega_data.append(lines)
    
    posi_num = len(posi_data)
    nega_num = len(nega_data)
    total_num = posi_num + nega_num
    # print(f"id:{id} nega_num:{nega_num} posi_num:{posi_num}")

    # data_list = []
    # 将data_list 写入新文件夹 "/home/wngys/lab/DeepFold/pair/train_pair_bool_90"
    f_w = open(newDir+id+".txt", "w")

    if total_num > batch_size: # 小于就忽略
        batch_num = math.floor((total_num - 1)/ batch_size) + 1
        # print(batch_num,end = ' ')
        left_num = total_num % batch_size # 没有用到?
        # print(left_num,end=' ')
        posi_num_batch = math.floor((posi_num - 1)/ batch_num) + 1
        # print(posi_num_batch,end = ' ')
        posi_left = posi_num % batch_num
        # print(posi_left, end = ' ')
        # 如果余数不为0 填充正例
        # print(batch_num-posi_left)

        if posi_num > batch_num-posi_left:
            posi_data.extend(posi_data[:(batch_num-posi_left)])
        else:
            cnt = 0
            tmp_data = copy.deepcopy(posi_data)
            while cnt < batch_num - posi_left:
                cnt += posi_num
                posi_data.extend(tmp_data)
                # print(len(tmp_data))

        nega_cusor = 0
        for i in range(batch_num):
            # data_list.extend(posi_data[i*posi_num_batch:(i+1)*posi_num_batch])
            for data in posi_data[i*posi_num_batch:(i+1)*posi_num_batch]:
                f_w.write(data)
            if i < batch_num - 1:
                # data_list.extend(nega_data[nega_cusor:nega_cusor+(batch_size-posi_num_batch)])
                for data in nega_data[nega_cusor:nega_cusor+(batch_size-posi_num_batch)]:
                    f_w.write(data)
                nega_cusor += batch_size-posi_num_batch
            else: # 最后一个分支 负例也填不满batch_size
                if batch_num - posi_left > batch_size - left_num:
                    # data_list.extend(nega_data[nega_cusor:nega_cusor+(batch_size-posi_num_batch)]) # 和前面batch处理相同，最后余出的负例不要了
                    for data in nega_data[nega_cusor:nega_cusor+(batch_size-posi_num_batch)]:
                        f_w.write(data)
                else:
                    # data_list.extend(nega_data[nega_cusor:])
                    for data in nega_data[nega_cusor:]:
                        f_w.write(data)

    else:
        # data_list.extend(posi_data)
        # data_list.extend(nega_data)
        for data in posi_data:
            f_w.write(data)
        for data in nega_data:
            f_w.write(data)
        
    f_w.close() 

In [1]:
import os

print(len(os.listdir("/home/wngys/lab/DeepFold/pair/train_pair_bool_90")))

8410


In [5]:
# 更改数据分布 确保每个batch至少一个正例
import math
import copy
import numpy as np
import random

theSelecTrainList = np.load("/home/wngys/lab/DeepFold/pair/train.npy", allow_pickle=True)

Dir = "/home/wngys/lab/DeepFold/pair/pair_bool_90/"
newDir = "/home/wngys/lab/DeepFold/pair/train_pair_bool_90/"
batch_size = 64

# print(theSelecTrainList)
idx = 0
for id in theSelecTrainList:
    idx += 1
    if idx % 100 == 0:
        print("idx: ", idx)
    posi_data = []
    nega_data = []

    with open(Dir+id+".txt", "r") as f_r:
        while True:
            lines = f_r.readline()
            if not lines:
                break
            # id2 = lines.split('\t')[0]
            flag = lines.split('\t')[1].split("\n")[0]
            # print(type(flag))
            if flag == "1":
                posi_data.append(lines)
            else:
                nega_data.append(lines)
    
    posi_num = len(posi_data)
    nega_num = len(nega_data)
    total_num = posi_num + nega_num
    # print(f"id:{id} nega_num:{nega_num} posi_num:{posi_num}")

    # data_list = []
    # 将data_list 写入新文件夹 "/home/wngys/lab/DeepFold/pair/train_pair_bool_90"
    f_w = open(newDir+id+".txt", "w")

    batch_num = math.floor((total_num - 1)/ batch_size) + 1

    Onebatch_pn = 6
    Onebatch_nn = 58

    last_posi_num = Onebatch_pn * batch_num
    last_nega_num = Onebatch_nn * batch_num

    tmp_pdata = copy.deepcopy(posi_data)
    while posi_num < last_posi_num:
        posi_data.extend(tmp_pdata)
        posi_num += len(tmp_pdata)

    tmp_ndata = copy.deepcopy(nega_data)
    while nega_num < last_nega_num:
        nega_data.extend(tmp_ndata)
        nega_num += len(tmp_ndata)

    random.shuffle(posi_data)
    random.shuffle(nega_data)

    for i in range(batch_num):
        for data in posi_data[i*6:(i+1)*6]:
            f_w.write(data)
        for data in nega_data[i*58:(i+1)*58]:
            f_w.write(data)

    f_w.close()


idx:  100
idx:  200
idx:  300
idx:  400
idx:  500
idx:  600
idx:  700
idx:  800
idx:  900
idx:  1000
idx:  1100
idx:  1200
idx:  1300
idx:  1400
idx:  1500
idx:  1600
idx:  1700
idx:  1800
idx:  1900
idx:  2000
idx:  2100
idx:  2200
idx:  2300
idx:  2400
idx:  2500
idx:  2600
idx:  2700
idx:  2800
idx:  2900
idx:  3000
idx:  3100
idx:  3200
idx:  3300
idx:  3400
idx:  3500
idx:  3600
idx:  3700
idx:  3800
idx:  3900
idx:  4000
idx:  4100
idx:  4200
idx:  4300
idx:  4400
idx:  4500
idx:  4600
idx:  4700
idx:  4800
idx:  4900
idx:  5000
idx:  5100
idx:  5200
idx:  5300
idx:  5400
idx:  5500
idx:  5600
idx:  5700
idx:  5800
idx:  5900
idx:  6000
idx:  6100
idx:  6200
idx:  6300
idx:  6400
idx:  6500
idx:  6600
idx:  6700
idx:  6800
idx:  6900
idx:  7000
idx:  7100
idx:  7200
idx:  7300
idx:  7400
idx:  7500
idx:  7600
idx:  7700
idx:  7800
idx:  7900
idx:  8000
idx:  8100
idx:  8200
idx:  8300
idx:  8400


In [3]:
il = [4, 3 ,54, 6]
import random
random.shuffle(il)
print(il)

[3, 6, 54, 4]
